In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

Mounted at /content/drive


In [ ]:
data = pd.read_csv("drive/MyDrive/DeepLearning/dataTrainnew.csv")

In [ ]:
X = data.iloc[:, [1, 2]].values
y = data.iloc[:, 3].values

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


In [ ]:
#y_encoded

In [ ]:
len(np.unique(y_encoded))

4

In [ ]:
# Xây dựng mô hình mạng neuron
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(len(np.unique(y_encoded)), activation='softmax')  # Lớp đầu ra với số lượng lớp bằng số loại khí
])

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))


Epoch 1/100
12/12 [==============================] - 3s 35ms/step - loss: 39.9352 - accuracy: 0.4930 - val_loss: 3.1149 - val_accuracy: 0.4111
Epoch 2/100
12/12 [==============================] - 0s 9ms/step - loss: 6.5202 - accuracy: 0.4706 - val_loss: 5.5526 - val_accuracy: 0.4222
Epoch 3/100
12/12 [==============================] - 0s 10ms/step - loss: 5.4223 - accuracy: 0.4986 - val_loss: 2.0062 - val_accuracy: 0.7556
Epoch 4/100
12/12 [==============================] - 0s 9ms/step - loss: 1.8568 - accuracy: 0.5238 - val_loss: 0.8994 - val_accuracy: 0.7222
Epoch 5/100
12/12 [==============================] - 0s 10ms/step - loss: 0.7853 - accuracy: 0.6779 - val_loss: 0.6908 - val_accuracy: 0.6222
Epoch 6/100
12/12 [==============================] - 0s 10ms/step - loss: 0.7034 - accuracy: 0.6667 - val_loss: 0.5463 - val_accuracy: 0.6889
Epoch 7/100
12/12 [==============================] - 0s 11ms/step - loss: 0.6098 - accuracy: 0.6807 - val_loss: 0.8808 - val_accuracy: 0.8222
Epoch 8

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

3/3 [==============================] - 0s 5ms/step - loss: 0.7958 - accuracy: 0.8333
Test Loss: 0.7957850694656372
Test Accuracy: 0.8333333134651184


In [ ]:
np.unique(y_encoded)

array([0, 1, 2, 3])

In [ ]:
np.unique(y)

array(['Dangerous', 'No Dangerous', 'Slightly dangerous',
       'Very dangerous'], dtype=object)

In [ ]:
#X_test

In [ ]:
predictions = model.predict(X_test)
predicted_labels = [np.argmax(prediction) for prediction in predictions]
# Tên của các lớp
class_names = ['Dangerous', 'No Dangerous', 'Slightly dangerous', 'Very dangerous']

# In ra nhãn dự đoán và nhãn thực tế dưới dạng tên lớp
for i in range(len(X_test)):
    predicted_label = class_names[predicted_labels[i]]
    actual_label = class_names[y_test[i]]
    print(f"Predicted Label: {predicted_label}, Actual Label: {actual_label}")


In [ ]:
# Lưu mô hình vào tệp
model.save('gas_detection_model.h5')

# Chuyển đổi mô hình sang định dạng TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Lưu mô hình TensorFlow Lite vào file
with open('gas_detection_model.tflite', 'wb') as f:
    f.write(tflite_model)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
tflite_model_name = 'gas_detection_model.tflite'  # Will be given .tflite suffix
c_model_name = 'model_data_gas'

In [ ]:
def hex_to_c_array(hex_data, var_name):

  c_str = ''

  # Create header guard
  c_str += '#ifndef ' + var_name.upper() + '_H\n'
  c_str += '#define ' + var_name.upper() + '_H\n\n'

  # Add array length at top of file
  c_str += '\nunsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'

  # Declare C variable
  c_str += 'unsigned char ' + var_name + '[] = {'
  hex_array = []
  for i, val in enumerate(hex_data) :

    # Construct string from hex
    hex_str = format(val, '#04x')

    # Add formatting so each line stays within 80 characters
    if (i + 1) < len(hex_data):
      hex_str += ','
    if (i + 1) % 12 == 0:
      hex_str += '\n '
    hex_array.append(hex_str)

  # Add closing brace
  c_str += '\n ' + format(' '.join(hex_array)) + '\n};\n\n'

  # Close out header guard
  c_str += '#endif //' + var_name.upper() + '_H'

  return c_str

In [ ]:
# Write TFLite model to a C source (or header) file
with open(c_model_name + '.h', 'w') as file:
  file.write(hex_to_c_array(tflite_model, c_model_name))